In [2]:
#!/usr/bin/env python
# coding: utf-8
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
#import optuna

import numpy as np
import torch
import torch.nn as nn
import os
#from tqdm import tqdm
#import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from torch.optim.lr_scheduler import CosineAnnealingLR
import math
import time
import joblib
warnings.filterwarnings("ignore", message="torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly.")
warnings.filterwarnings('ignore',message='None of the inputs have requires_grad=True. Gradients will be None')
warnings.simplefilter(action='ignore', category=FutureWarning)
train_df=pd.read_csv("./energy-anomaly-detection/train.csv")

#from momentfm.utils.anomaly_detection_metrics import adjbestf1

In [3]:
pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 76.0 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 96.9 MB/s eta 0:00:00
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib]7 [matplotlib]
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [6]:
class TimeDataset(Dataset):
    def __init__(self,df):
        super().__init__()
        self.df=df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        row=self.df.iloc[idx]
        return row.meter_reading,1,row.anomaly

In [7]:
DEVICE=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
unique_ids=train_df['building_id'].unique()
df_list=[]

/home/ai-iot-1/Downloads/Kajeeth/.conda/lib/python3.11/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [8]:
for b_id in unique_ids:

    building_df=train_df[train_df['building_id']==b_id].copy(deep=True)
    building_df['meter_reading']=building_df['meter_reading'].replace(float('nan'),
                                                                     building_df['meter_reading'].median())
    scaler=StandardScaler()
    building_df['meter_reading']=scaler.fit_transform(pd.DataFrame(building_df['meter_reading']))
    building_df.reset_index(drop=True,inplace=True)
    df_list.append(building_df)
print(len(unique_ids))

200


In [9]:
imputed_train=pd.concat(df_list)

In [10]:
class VarEncoderDecoder(nn.Module):
    def __init__(self,hidden_layers,hidden_size,latent_dim=32,seq_length=512):
        super().__init__()
        
        hidden_sizes=[]
        hidden_sizes.append(seq_length)
        for i in range(hidden_layers):
            hidden_sizes.append(hidden_size)
            hidden_size//=2
        
        self.encoder=nn.ModuleList()
        for i in range(1,len(hidden_sizes)):
            linear=nn.Linear(hidden_sizes[i-1],hidden_sizes[i])
            activation=nn.ReLU()
            norm=nn.BatchNorm1d(hidden_sizes[i])
            self.encoder.append(linear)
            self.encoder.append(norm)
            self.encoder.append(activation)

        self.encoder=nn.Sequential(*self.encoder)
        
        #mu,var
        self.enc_fc_mu=nn.Linear(hidden_sizes[-1],latent_dim)
        self.enc_fc_var=nn.Linear(hidden_sizes[-1],latent_dim)

        self.decoder_in=nn.Linear(latent_dim,hidden_sizes[-1])
        self.decoder=nn.ModuleList()
        hidden_sizes=list(reversed(hidden_sizes))
        for i in range(1,len(hidden_sizes)):
            linear=nn.Linear(hidden_sizes[i-1],hidden_sizes[i])
            activation=nn.LeakyReLU()
            norm=nn.BatchNorm1d(hidden_sizes[i])
            self.decoder.append(linear)
            self.decoder.append(norm)
            self.decoder.append(activation)        
        self.decoder=nn.Sequential(*self.decoder)
        
    def reparameterize(self, mu, logvar):
        # Reparameterization trick
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def forward(self,X):
        X=self.encoder(X)
        # print(X.shape)
        mu=self.enc_fc_mu(X)
        log_var=self.enc_fc_var(X)
        X=self.reparameterize(mu,log_var)
        X=self.decoder_in(X)
        # print(X.shape)
        X=self.decoder(X)
        # print(X.shape)
        return X

In [11]:
def train_one_epoch(df_train,model,optimizer,loss_func,train_ids,
                   CONTEXT_LEN,BATCH_SIZE):
    model.train()
    
    losses=[]
    list_of_loaders=[]
    list_of_medians=[]
    for _id in train_ids:
        building=df_train[df_train['building_id']==_id].copy(deep=True)
        building.reset_index(drop=True,inplace=True)
        median=building['meter_reading'].median()
        dataset=TimeDataset(building)
        loader=DataLoader(dataset,batch_size=CONTEXT_LEN,shuffle=False,drop_last=True)
        iter_loader=iter(loader)
        list_of_loaders.append(iter_loader)
        list_of_medians.append(median)


    min_len=14
    for it in tqdm(range(min_len)):
        batch_x=[]
        batch_target=[]
        c=0
        encoder_hidden=None
        for ix,loader in enumerate(list_of_loaders):
            single_x,_,labels=next(loader)
            median=list_of_medians[ix]
            labels=labels.bool()
            target_x=single_x.clone().detach()
            target_x[labels]=median
            target_x=torch.unsqueeze(target_x,dim=0)
            batch_target.append(target_x)
            single_x=torch.unsqueeze(single_x,dim=0)
            batch_x.append(single_x)
            c+=1
            if (c!=len(list_of_loaders)) and (c % BATCH_SIZE != 0):
                continue
            batch_x=torch.cat(batch_x,dim=0).double().to(DEVICE)
            batch_target=torch.cat(batch_target,dim=0).double().to(DEVICE)
            output = model(batch_x) 
            loss = loss_func(output, batch_target)
            if math.isnan(loss.item()):
                print("Nan acquired! Quitting!")
                break
            losses.append(loss)
            batch_x=[]
            batch_target=[]
            # Backward
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=5.0)
            optimizer.step()
    print(f"Training Loss: ",torch.mean(torch.tensor(losses)))
    return torch.mean(torch.tensor(losses))   

In [12]:
def test_model(df_val,model,val_ids):
    model.eval()
    all_scores=[]
    list_of_loaders=[]
    for b_id in val_ids:
        trues=[]
        preds=[]
        labels=[]
        first_building=df_val[df_val['building_id']==b_id].copy(deep=True)
        first_building.reset_index(drop=True,inplace=True)
        first_dataset=TimeDataset(first_building)
        loader=DataLoader(first_dataset, batch_size=512, shuffle=False, drop_last=False)
        with torch.no_grad():
            for batch_x, _,batch_labels in tqdm(loader, total=len(loader)):
                batch_x = batch_x.to(DEVICE).double()
                length=len(batch_x)
                if length<512:
                    zeros=torch.zeros(512-length)
                    zeros=zeros.to(DEVICE)
                    batch_x=torch.cat([batch_x,zeros],dim=0)
                    
                batch_x=torch.unsqueeze(batch_x,dim=0)
                batch_x=batch_x.to(DEVICE)
                output= model(batch_x) # [batch_size, window_size]
    
                trues.extend(batch_x.detach().squeeze().cpu().numpy()[:length])
                preds.extend(output.squeeze().cpu().numpy()[:length])
                labels.extend(batch_labels.detach().cpu().numpy())
            trues=np.array(trues)
            preds=np.array(preds)
            labels=np.array(labels)
            anomaly_scores=(trues-preds)**2
            score=adjbestf1(y_true=labels, y_scores=anomaly_scores)
            all_scores.append(score)
    return np.mean(all_scores)

In [13]:
import os
os.makedirs('./LEAD/validation_folds_VAE/', exist_ok=True)

In [14]:
unique_ids = train_df['building_id'].unique()
unique_ids

array([   1,   32,   41,   55,   69,   79,   82,   91,  107,  108,  111,
        112,  117,  118,  119,  121,  136,  137,  139,  141,  144,  147,
        148,  149,  159,  171,  173,  174,  181,  183,  190,  235,  238,
        240,  246,  247,  248,  253,  254,  263,  270,  275,  276,  278,
        290,  293,  312,  318,  335,  345,  356,  423,  439,  492,  534,
        560,  623,  653,  657,  658,  666,  667,  673,  675,  677,  680,
        683,  685,  687,  693,  697,  698,  701,  708,  710,  721,  722,
        729,  730,  732,  739,  742,  801,  827,  844,  848,  879,  880,
        881,  882,  884,  886,  887,  889,  890,  892,  893,  894,  895,
        896,  903,  905,  909,  914,  919,  922,  924,  925,  926,  928,
        929,  931,  935,  936,  941,  942,  945,  948,  950,  952,  961,
        966,  967,  968,  969,  970,  971,  973,  974,  975,  977,  978,
        981,  988,  990,  992,  994,  996, 1001, 1007, 1068, 1073, 1074,
       1106, 1120, 1128, 1137, 1141, 1143, 1147, 11

In [15]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for i, (_, val_index) in enumerate(kf.split(unique_ids)):
    val_ids = unique_ids[val_index]
    joblib.dump(val_ids, f"./LEAD/validation_folds_VAE/val_id_fold{i}.pkl")
    print(f"saved fold {i} with {len(val_ids)} validation IDs")

saved fold 0 with 40 validation IDs
saved fold 1 with 40 validation IDs
saved fold 2 with 40 validation IDs
saved fold 3 with 40 validation IDs
saved fold 4 with 40 validation IDs


In [16]:
from tqdm import tqdm

In [17]:
def single_run(lr,epochs,beta_1,beta_2,hidden_size,num_layers,latent_dim):
    EPOCHS=epochs
    LR=lr
    TOTAL_LEN=365*24
    INPUT_SIZE=512
    CONTEXT_LEN=INPUT_SIZE
    HIDDEN_SIZE=hidden_size
    NUM_LAYERS=num_layers
    LATENT_DIM=latent_dim
    BATCH_SIZE=160
    scores=[]
    n_splits=5
    id_path="./LEAD/validation_folds_VAE/"
    for i in range(n_splits):
        val_ids=joblib.load(id_path+f"val_id_fold{i}.pkl")
        all_ids=imputed_train.building_id.unique()
        train_ids=[x for x in all_ids if x not in val_ids]
        print(f"FOLD: {i}")
        print("*"*20)
        model=VarEncoderDecoder(NUM_LAYERS,HIDDEN_SIZE,latent_dim=LATENT_DIM)
        model = model.to(DEVICE).double()
    
    
        df_train=imputed_train[~imputed_train.building_id.apply(lambda x: x in val_ids)]
        df_val=imputed_train[imputed_train.building_id.apply(lambda x: x  in val_ids)]
        loss_func=torch.nn.MSELoss()
        optimizer=torch.optim.Adam(model.parameters(),lr=LR,betas=(beta_1,beta_2))
        T_MAX=int((TOTAL_LEN/CONTEXT_LEN)/BATCH_SIZE*EPOCHS)
        val_score=0
        print(f"Starting f1_score={val_score}")
        print("TRAINING")
        print("*"*50)
        cur_score=0
        for epoch in range(EPOCHS):
            train_loss=train_one_epoch(df_train,model,optimizer,loss_func,train_ids,CONTEXT_LEN,BATCH_SIZE)
            val_score=test_model(df_val,model,val_ids)
            print(f"EPOCH= {epoch} Train Loss(MSE)={train_loss} Val Score(F1)= {val_score}")
            if val_score>cur_score:
                cur_score=val_score
                print(f'current best={cur_score}')
        
        scores.append(cur_score)
    return scores

In [18]:
def vaed_objective(trial):
    lr=trial.suggest_float('learning_rate', 1e-6, 1e-1)
    epochs=trial.suggest_int('epochs', 5,50)
    beta_1=trial.suggest_float('beta_1',0.1,1)
    beta_2=trial.suggest_float('beta_2',0.1,1)
    hidden_size=trial.suggest_categorical('hidden_size',[256,192,164,128,64])
    latent_dim=trial.suggest_categorical('latent_dim',[32,48,64,92])
    num_layers=trial.suggest_int('num_layers',1,1)
    scores=single_run(lr,epochs,beta_1,beta_2,hidden_size,num_layers,latent_dim)
    return np.mean(scores)

In [19]:
import optuna

/home/ai-iot-1/Downloads/Kajeeth/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
from momentfm.utils.anomaly_detection_metrics import adjbestf1

In [ ]:
start_time = time.time()
study_vaed = optuna.create_study(direction='maximize')
study_vaed.optimize(vaed_objective, n_trials=50)
end_time = time.time()
elapsed_time_vaed = end_time - start_time
print(f"VAED training took {elapsed_time_vaed:.2f} seconds.")

[I 2025-08-07 17:32:00,123] A new study created in memory with name: no-name-25538ed0-d7b7-4f1a-a4ec-c499341c912e


FOLD: 0
********************
Starting f1_score=0
TRAINING
**************************************************


100%|██████████| 14/14 [00:42<00:00,  3.00s/it]


Training Loss:  tensor(1.0403, dtype=torch.float64)


100%|██████████| 18/18 [00:00<00:00, 26.67it/s]


...

100%|██████████| 14/14 [00:45<00:00,  3.25s/it]


Training Loss:  tensor(0.6519, dtype=torch.float64)


100%|██████████| 18/18 [00:00<00:00, 25.34it/s]


EPOCH= 10 Train Loss(MSE)=0.6518824517548035 Val Score(F1)= 0.6015064027912386
current best=0.6015064027912386


 50%|█████     | 7/14 [00:21<00:20,  2.86s/it]

In [ ]:
best_params=study_vaed.best_trial.params
print(best_params)

joblib.dump(best_params,'best_variational_autoencoderdecoder_hyper_params.pkl')
joblib.dump(study_vaed,'vaed_study.pkl')
